# Modelo de Aprendizaje

El modelo de aprendizaje elegido, será un sistema de recomendación item-item, en el cual al ingresar el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Se nos hizo la recomendación de aplicar la similitud del coseno, entonces voy a rescatar del dataset original la columna que tenía los reviews de los usuarios. Esta columna fue eliminada del dataset después de utilizarla para calcular el Sentiment Analysis.

Haré entonces un proceso similar al realizado en el ETL de el archivo users:reviews.json, pero conservando la columna de los reviews

In [23]:
import pandas as pd
import ast
import warnings
warnings.filterwarnings('ignore')

A continuación se crea una función que ayudará con la carga de la información en DataFrames de Pandas

In [5]:
def cargar_df(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    rows = []
    with open(ruta, 'r', encoding='utf-8') as file:                         # se lee el archivo iterando línea por línea y agregando a lista vacia
        for line in file:
            rows.append(ast.literal_eval(line)) 
    
    df = pd.DataFrame(rows)                                                 # se carga la info en un df de Pandas            
    df = df.explode(variable_anidada).reset_index()                         # se separan en filas los datos anidados y se resetea index
    df = df.drop(columns="index")                                           # se elimina el indice original
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   # se realiza la apertura en columnas de la informacion anidada
    df = df.drop(columns=variable_anidada)                                  # se elimina la columna anidada original

    return df

In [24]:
df_reviews = cargar_df(r'Dataset/user_reviews.json', "reviews")
df_reviews.shape

(59333, 9)

In [25]:
# Elimino valores nulos
df_reviews = df_reviews.dropna().reset_index(drop=True)

#Elimino valores duplicados
df_reviews = df_reviews.drop_duplicates()

#Como sólo me interesan las columnas item_id y review, elimino las demás
df_reviews = df_reviews.drop(columns=['user_id', 'user_url', 'funny', 'posted', 
                                     'last_edited', 'helpful', 'recommend'])

In [26]:
#Modifico tipos de datos
df_reviews['review']=df_reviews['review'].astype(str)
df_reviews['item_id']=df_reviews['item_id'].astype(float)

Ahora necesito los nombres de los juegos, entonces voy a cargar el archivo steam games (en este caso sí usaré el archivo resultante del ETL), para extraer de este los nombres de los juegos

In [27]:
# Leo el archivo steam_games.parquet y lo guardo en un dataframe df_games
df_games = pd.read_parquet('steam_games.parquet')

In [28]:
#Tomo sólo las columnas que necesito
df_games_modelo = df_games[['Name', 'Game_id']]

#cambio los nombres de las columnas
df_games_modelo.columns = ['item_name', 'item_id']

#Aseguro los tipos de datos
df_games_modelo['item_name']=df_games_modelo['item_name'].astype(str)
df_games_modelo['item_id']=df_games_modelo['item_id'].astype(float)

In [36]:
#Combino ambos dataframes
df_modelo = pd.merge(df_reviews, df_games_modelo, on="item_id", how = 'inner')
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48949 entries, 0 to 48948
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_id    48949 non-null  float64
 1   review     48949 non-null  object 
 2   item_name  48949 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.1+ MB


In [43]:
#Como sólo necesito el nombre del juego, puedo eliminar la columna item_id
#df_modelo = df_modelo.drop(columns = 'item_id')
df_modelo

,item_id,review,item_name
0,1250.0,Simple yet with great replayability. In my opi...,Killing Floor
1,1250.0,"Amazing, Non-stop action of blowing stuff to b...",Killing Floor
2,1250.0,"Compared to Left 4 Dead 2, this game REALLY gi...",Killing Floor
3,1250.0,Jogo ♥♥♥♥.,Killing Floor
4,1250.0,cara nas imagens esse jogo da pouco de medo ma...,Killing Floor
...,...,...,...
48944,307130.0,"Asteria is a fast paced indie platformer, wrap...",Asteria
48945,209120.0,"Great game, awkward to get running in windows 10",Street Fighter X Tekken
48946,220090.0,GET THIS GAME AND CHAPTER TWO!!!!!!!!!!!!!!!!!...,The Journey Down: Chapter One
48947,262850.0,THIS GAME!!!!!!!!!!!!!!!!!! WOOOOOOOOOOOOOOOOO...,The Journey Down: Chapter Two


In [40]:
# Ahora que ya tengo listo el dataset para el modelo lo guardo en un archivo parquet
df_modelo.to_parquet('dataset_ml.parquet', engine="pyarrow")

A continuación desarrollaré el código para el modelo de recomendación. Este código será probado en este notebook, pero deberá ser insertado en el archivo que se suba a render para el deploy, por eso

In [111]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

df_modelo = pd.read_parquet('dataset_ml.parquet')

In [114]:
def recomendacion_juego( id : float): 
    """ Ingresando el id de producto, deberíamos recibir una lista con 5 juegos 
    recomendados similares al ingresado. """

    #tomo un dataframe auxiliar sólo con las columnas reviews y item_id
    #df = df_modelo.drop(columns = 'item_name')

    #agrupo los reviews por juegos
    df_modelo["review"] = df_modelo["review"].fillna("")
    grouped = df.groupby('item_id').agg(lambda x: ' '.join(x)).reset_index()
    
    #Vectorización de términos mediante tf-idf, usando stop words en inglés
    tfidf_vectorizer = TfidfVectorizer(stop_words = 'english')

    #Aplica el vectorizador tf-idf a la columna 'review' del dataframe agrupado
    tfidf_matrix = tfidf_vectorizer.fit_transform(grouped['review'])

    #Calcula la similitud del coseno entre las reseñas utilizando linear_kernel
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    #busca el indice del item de entrada de la función
    idx = grouped.index[grouped['item_id']== id].tolist()[0]

    #Se crea una lista de tuples, donde cada tuple contiene el índice del artículo 
    #y su puntaje de similitud de coseno con el artículo específico.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Se ordena descendente, de manera que las reseñas más similares aparezcan primero.
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

    #selecciono las 5 primeras
    sim_scores = sim_scores[1:6]
    item_indices = [i[0] for i in sim_scores]
    
    #obtiene la lista de item_id correspondiente a los índices
    salida_ids = grouped['item_id'].iloc[item_indices].tolist()

    #obtiene la lista de nombres
    salida_nombres = df_modelo.loc[df['item_id'].isin(salida_ids),'item_name'].unique().tolist()
    
    return salida_nombres

In [115]:
recomendacion_juego (307130)

['Alien Swarm',
 'Wyv and Keep: The Temple of the Lost Idol',
 'Massive',
 'NyxQuest: Kindred Spirits',
 'Lost Planet® 2']